In [1]:
import os
from glob import glob
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
df

,staID,DT,value,var,bad
0,161,1992-06-03 17:00:00,0.00,RF,False
1,161,1992-06-03 17:00:00,94.30,RH_1,False
2,161,1992-06-03 17:00:00,454.80,SWup,False
3,161,1992-06-03 17:00:00,11.24,Tair_1,False
4,161,1992-06-03 18:00:00,0.00,RF,False
...,...,...,...,...,...
886738,161,2021-08-21 13:00:00,18.25,Tair_1,False
886739,161,2021-08-21 14:00:00,0.00,RF,False
886740,161,2021-08-21 14:00:00,16.76,RH_1,False
886741,161,2021-08-21 14:00:00,1109.21,SWup,False


In [10]:
csv_fns = glob(os.path.join('data', 'raw', '*.csv'))

overall_bad_count = 0
overall_good_count = 0

for i, fn in enumerate(csv_fns):
    df = pd.read_csv(fn)
    df = df[df['DT'].str.contains(":")]
    df['DT'] = pd.to_datetime(df['DT'])
    bad_count = df[df['bad'] == True]['bad'].count()
    overall_bad_count += bad_count
    good_count = df[df['bad'] == False]['bad'].count()
    overall_good_count += good_count

    if i % 2 == 0:
        # if even then take FIRST 4 years as held out test data
        test_df = df[df['DT'].dt.year <= 1996]
        train_df = df[df['DT'].dt.year > 1996]
    else:
        # if odd take the LAST 4 years as training data
        test_df = df[df['DT'].dt.year >= 2018]
        train_df = df[df['DT'].dt.year < 2018]

    assert (df.shape[0] == test_df.shape[0] + train_df.shape[0])

    
    test_fn = os.path.join('data', 'test', f'test_{os.path.basename(fn)}')
    test_df.to_csv(test_fn, index=False)
    train_fn = os.path.join('data', 'train', f'train_{os.path.basename(fn)}')
    train_df.to_csv(train_fn, index=False)

    # plt.plot(df['DT'], df['bad'])
    # plt.show()
    
    # display(df)

print(f'Bad count {overall_bad_count}, good count {overall_good_count}')

Bad count 82665, good count 3766465


# Load data for site wise (leave-one-out) models
For this we just 

In [ ]:
df_list = [
    pd.read_csv(r'https://raw.githubusercontent.com/jnicolow/anomaly_detection/refs/heads/main/data/raw/sta119.csv'),
    pd.read_csv(r'https://raw.githubusercontent.com/jnicolow/anomaly_detection/refs/heads/main/data/raw/sta151.csv'),
    pd.read_csv(r'https://raw.githubusercontent.com/jnicolow/anomaly_detection/refs/heads/main/data/raw/sta152.csv'),
    pd.read_csv(r'https://raw.githubusercontent.com/jnicolow/anomaly_detection/refs/heads/main/data/raw/sta161.csv')
]

for i in range(len)

# Load data for site-specific models